# Importing Libraries 

In [1]:
import logging 
#Importing libraries 
logging.basicConfig(filename='Log_Example.log',level=logging.DEBUG)
logging.info("Libraries importing started")
import numpy as np
import pandas as pd
import cx_Oracle
import dateutil
import datetime as dt 
import xgboost as xgb
import lightgbm as lgb
import seaborn as sns
color = sns.color_palette()
import matplotlib
import matplotlib.pyplot as plt 
from datetime import datetime as dt 
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
logging.info("Libraries importing completed")

# Connecting to 172 AND 182 

In [2]:
logger=logging.getLogger(__name__)
def dbConnection(connectString):
    logging.info("DB connection started %s",connectString)
    connection = cx_Oracle.connect(connectString)
    logger.info("DB connection successfull %s",connection)
    logging.info
    return connection 

connectString_182='anlinq/anlinq@172.16.1.182:1521/dwh'
connectString_172='anlinq/anlinq@172.16.1.172:1521/BIDB'

connection_182=dbConnection(connectString_182)
connection_172=dbConnection(connectString_172)


# Vishwa Data Extraction

In [6]:
#SQL Queries 

--Customer 
select a.USER_ID,CUST_ACCOUNT,NIC_NUMBER,CUST_REF_NO, BANK_CUST_FLG,CUST_GENDER AS V_GENDER , 
CIVIL_STATUS AS V_CIVIL_STATUS, 
TO_CHAR(CUST_DOB) AS V_CUST_DOB, 
COMM_CITY AS V_CITY,CHARGE_PROFILE, 
case when CUST_CCARD is not null then 'Yes' else 'No' END as CREDIT_CARD_FLAG, 
case when lower(comm_country) like '%sri%' then 'Yes' else 'No' end as V_COUNTRY_SL, 
round((LAST_DAY(add_months(trunc(sysdate,'mm'),-1))-activate_date),0) as VISHWA_AGE, 
round((LAST_DAY(add_months(trunc(sysdate,'mm'),-1))-last_login_date),0) as LAST_LOGIN_AGE 
from TBAADM.svr_user_master_table a 
WHERE R_STATUS=104 

--Transfer 
select USER_ID,sum(TXN_AMOUNT) as V_TXN_AMOUNT,sum(TXN_CURR_AMOUNT) AS V_TXN_CURR_AMOUNT,sum(TXN_AMOUNT_LKR) AS V_TXN_AMOUNT_LKR,count(*)AS V_TXN_CNT 
from tbaadm.svr_transfers_master_table
where tran_date>= add_months(trunc(sysdate,'mm'),-6) AND tran_date<= LAST_DAY(add_months(trunc(sysdate,'mm'),-1))
group by USER_ID
ORDER BY USER_ID

--Payments
SELECT CUST_ID as USER_ID,sum(AMOUNT) AS V_PMNT_AMOUNT,sum(TRANSACTION_AMT_LKR) AS V_PMNT_TRANSACTION_AMT_LKR,count(*) AS V_PMNT_COUNT 
FROM tbaadm.svr_payments_master_table
where tran_date>= add_months(trunc(sysdate,'mm'),-6) AND tran_date<= LAST_DAY(add_months(trunc(sysdate,'mm'),-1))
group by CUST_ID
ORDER BY USER_ID

--Scheduled payments 

SELECT 
  USER_ID, sum(TXN_AMOUNT_LKR) AS V_SCH_PMNT_AMOUNT_LKR,sum(CREDIT_AMOUNT) AS V_SCH_PMNT_CREDIT_AMOUNT,sum(CREDIT_CURR_AMOUNT) AS V_SCH_PMNT_CREDIT_CURR_AMOUNT
  ,count(*) V_SCH_PMNT_CNT
FROM tbaadm.svr_schedule_pmt_tfr_table
where SCH_DATE>= add_months(trunc(sysdate,'mm'),-6) AND SCH_DATE<= LAST_DAY(add_months(trunc(sysdate,'mm'),-1))
group by USER_ID
ORDER BY USER_ID

<cx_Oracle.Connection to anlinq@172.16.1.172:1521/BIDB>

In [4]:
vishCustomer='select a.USER_ID,CUST_ACCOUNT,NIC_NUMBER,CUST_REF_NO, BANK_CUST_FLG,CUST_GENDER AS V_GENDER , \
CIVIL_STATUS AS V_CIVIL_STATUS, \
TO_CHAR(CUST_DOB) AS V_CUST_DOB, \
COMM_CITY AS V_CITY,CHARGE_PROFILE, \
case when CUST_CCARD is not null then \'Yes\' else \'No\' END as CREDIT_CARD_FLAG, \
case when lower(comm_country) like \'%sri%\' then \'Yes\' else \'No\' end as V_COUNTRY_SL, \
round((LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-activate_date),0) as VISHWA_AGE, \
round((LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-last_login_date),0) as LAST_LOGIN_AGE \
from TBAADM.svr_user_master_table a \
WHERE R_STATUS=104 '
vishCustomer

"select a.USER_ID,CUST_ACCOUNT,NIC_NUMBER,CUST_REF_NO, BANK_CUST_FLG,CUST_GENDER AS V_GENDER , CIVIL_STATUS AS V_CIVIL_STATUS, TO_CHAR(CUST_DOB) AS V_CUST_DOB, COMM_CITY AS V_CITY,CHARGE_PROFILE, case when CUST_CCARD is not null then 'Yes' else 'No' END as CREDIT_CARD_FLAG, case when lower(comm_country) like '%sri%' then 'Yes' else 'No' end as V_COUNTRY_SL, round((LAST_DAY(add_months(trunc(sysdate,'mm'),-1))-activate_date),0) as VISHWA_AGE, round((LAST_DAY(add_months(trunc(sysdate,'mm'),-1))-last_login_date),0) as LAST_LOGIN_AGE from TBAADM.svr_user_master_table a WHERE R_STATUS=104 "

In [5]:
#Extracting all vishwa customer data 
def get_raw_vishwaCustomer(connection,queryString):
    logging.info("vishwa customer data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("vishwa customer data extraction completed")
    return df 

vishwaCustomer=get_raw_vishwaCustomer(connection_182,vishCustomer)

In [50]:
vishwaCustomer.head(3)

,USER_ID,CUST_ACCOUNT,NIC_NUMBER,CUST_REF_NO,BANK_CUST_FLG,V_GENDER,V_CIVIL_STATUS,V_CUST_DOB,V_CITY,CHARGE_PROFILE,CREDIT_CARD_FLAG,V_COUNTRY_SL,VISHWA_AGE,LAST_LOGIN_AGE
0,menake,1.001e+11,761160532v,929100000003,Y,M,Married,25-04-1976,Thalahena,STF,No,Yes,1406.0,-15.0
1,wiggin,1.00105e+11,862102045V,929100000008,Y,M,Single,28-07-1986,kandy,STF,No,Yes,2597.0,-16.0
2,manjula,1.001e+11,732110577V,929100000010,Y,M,Married,29-07-1973,Ragama,STF,No,Yes,2497.0,-9.0


In [6]:
vishTransfer='select USER_ID, \
SUM(CASE WHEN TRAN_MODE=\'OT\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_OT_TXN_AMOUNT_LKR, \
SUM(CASE WHEN TRAN_MODE=\'RA\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_RA_TXN_AMOUNT_LKR, \
SUM(CASE WHEN TRAN_MODE=\'RT\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_RT_TXN_AMOUNT_LKR, \
SUM(CASE WHEN TRAN_MODE=\'MC\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_MC_TXN_AMOUNT_LKR, \
SUM(CASE WHEN TRAN_MODE=\'TT\' then TXN_AMOUNT_LKR ELSE 0 END) AS V_TT_TXN_AMOUNT_LKR \
,count(*)AS V_TXN_CNT \
from tbaadm.svr_transfers_master_table \
where tran_date>= add_months(trunc(sysdate,\'mm\'),-6) AND tran_date<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
group by USER_ID \
ORDER BY USER_ID'
vishTransfer

"select USER_ID, SUM(CASE WHEN TRAN_MODE='OT' then TXN_AMOUNT_LKR ELSE 0 END) AS V_OT_TXN_AMOUNT_LKR, SUM(CASE WHEN TRAN_MODE='RA' then TXN_AMOUNT_LKR ELSE 0 END) AS V_RA_TXN_AMOUNT_LKR, SUM(CASE WHEN TRAN_MODE='RT' then TXN_AMOUNT_LKR ELSE 0 END) AS V_RT_TXN_AMOUNT_LKR, SUM(CASE WHEN TRAN_MODE='MC' then TXN_AMOUNT_LKR ELSE 0 END) AS V_MC_TXN_AMOUNT_LKR, SUM(CASE WHEN TRAN_MODE='TT' then TXN_AMOUNT_LKR ELSE 0 END) AS V_TT_TXN_AMOUNT_LKR ,count(*)AS V_TXN_CNT from tbaadm.svr_transfers_master_table where tran_date>= add_months(trunc(sysdate,'mm'),-6) AND tran_date<= LAST_DAY(add_months(trunc(sysdate,'mm'),-1)) group by USER_ID ORDER BY USER_ID"

In [7]:
#Extracting all vishwa transfer data 
def get_raw_vishwaTransfer(connection,queryString):
    logging.info("vishwa transfer data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("vishwa transfer data extraction completed")
    return df 

vishwaTransfer=get_raw_vishwaTransfer(connection_182,vishTransfer)

In [48]:
vishwaTransfer.head(3)

,USER_ID,V_OT_TXN_AMOUNT_LKR,V_RA_TXN_AMOUNT_LKR,V_RT_TXN_AMOUNT_LKR,V_MC_TXN_AMOUNT_LKR,V_TT_TXN_AMOUNT_LKR,V_TXN_CNT
0,0000979141,0.0,327600.0,0,0,10000.00,15
1,0001000016200,911000.0,10050.0,0,0,98755.29,14
2,000150125588,2111000.0,745800.0,0,0,15000.00,50


In [8]:
vishPayment='SELECT CUST_ID as USER_ID,sum(TRANSACTION_AMT_LKR) AS V_PMNT_TRANSACTION_AMT_LKR,count(*) AS V_PMNT_COUNT \
FROM tbaadm.svr_payments_master_table \
where tran_date>= add_months(trunc(sysdate,\'mm\'),-6) AND tran_date<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
group by CUST_ID \
ORDER BY USER_ID'
vishPayment

"SELECT CUST_ID as USER_ID,sum(TRANSACTION_AMT_LKR) AS V_PMNT_TRANSACTION_AMT_LKR,count(*) AS V_PMNT_COUNT FROM tbaadm.svr_payments_master_table where tran_date>= add_months(trunc(sysdate,'mm'),-6) AND tran_date<= LAST_DAY(add_months(trunc(sysdate,'mm'),-1)) group by CUST_ID ORDER BY USER_ID"

In [9]:
#Extracting all vishwa payment data 
def get_raw_vishwaPayment(connection,queryString):
    logging.info("vishwa payment data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("vishwa payment data extraction completed")
    return df 

vishwaPayment=get_raw_vishwaPayment(connection_182,vishPayment)

In [49]:
vishwaPayment.head(3)

,USER_ID,V_PMNT_TRANSACTION_AMT_LKR,V_PMNT_COUNT
0,0000979141,2945.00,30
1,0001000016200,36553.89,18
2,000150125588,407806.77,10


In [10]:
vishSchdPayment='SELECT USER_ID, sum(TXN_AMOUNT_LKR) AS V_SCH_PMNT_AMOUNT_LKR,count(*) V_SCH_PMNT_CNT \
FROM tbaadm.svr_schedule_pmt_tfr_table \
where SCH_DATE>= add_months(trunc(sysdate,\'mm\'),-6) AND SCH_DATE<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
group by USER_ID \
ORDER BY USER_ID'
vishSchdPayment

"SELECT USER_ID, sum(TXN_AMOUNT_LKR) AS V_SCH_PMNT_AMOUNT_LKR,count(*) V_SCH_PMNT_CNT FROM tbaadm.svr_schedule_pmt_tfr_table where SCH_DATE>= add_months(trunc(sysdate,'mm'),-6) AND SCH_DATE<= LAST_DAY(add_months(trunc(sysdate,'mm'),-1)) group by USER_ID ORDER BY USER_ID"

In [11]:
#Extracting all vishwa sheduled payment data 
def get_raw_vishwaSchdPayment(connection,queryString):
    logging.info("vishwa Scheduled  payment data extraction started")
    df = pd.read_sql(
        queryString,con=connection)
    logging.info("vishwa Scheduled payment data extraction completed")
    return df 

vishwaSchdPayment=get_raw_vishwaSchdPayment(connection_182,vishSchdPayment)
#print(vishwa.head(5))

In [12]:
vishwaSchdPayment.head(3)

,USER_ID,V_SCH_PMNT_AMOUNT_LKR,V_SCH_PMNT_CNT
0,0001000016200,89212.73,7
1,001000001050,1156874.00,12
2,001000001220,0.00,1


In [147]:

#Extracting the user_id whose have only one cust_id
vishDisUser11='select distinct user_id \
from tbaadm.svr_cust_id_table \
where status_id=104 \
group by user_id \
having count(cust_id)= 1'


#Extracting the user_id whose have many cust_id
vishDisUser1M='select distinct user_id \
from tbaadm.svr_cust_id_table \
where status_id=104 \
group by user_id \
having count(cust_id)>1'


#Extracting the cust_id whose have only one user_id
vishDisCust11='select distinct cust_id \
from tbaadm.svr_cust_id_table \
where status_id=104 \
group by cust_id \
having count(user_id)=1'

#Extracting the cust_id whose have  many user_ids
vishDisCust1M='select distinct cust_id \
from tbaadm.svr_cust_id_table \
where status_id=104 \
group by cust_id \
having count(user_id)>1'

#extracting user_id and cust_id pairs(This contain many to many relationship structure )
vishUserCust='select distinct user_id,cust_id \
from tbaadm.svr_cust_id_table \
where status_id=104'

vishDisUser

'select distinct user_id from tbaadm.svr_cust_id_table group by user_id having count(cust_id)= 1'

In [271]:
#Extracting all vishwa user cust track data 
#Note: Only one to one mapping between cust_id and user id recodrs are considered at the moment. 
def get_raw_vishwaUserCustTrack(connection,queryString1,queryString2,queryString3,queryString4,queryString5):
    logging.info("User_id cust_id mapping data extraction started")
    UC11 = pd.read_sql(queryString1,con=connection_182)
    UC1M = pd.read_sql(queryString2,con=connection_182)
    CU11 = pd.read_sql(queryString4,con=connection_182)
    CU1M = pd.read_sql(queryString5,con=connection_182)
    df = pd.read_sql(queryString3,con=connection_182)
    
    dfUC11=df.merge(right=UC11.reset_index(drop=True), how='inner', on='USER_ID')
    
    d=dfUC11.merge(right=CU11.reset_index(drop=True), how='inner', on='CUST_ID')
    d1 = df.merge(right=UC1M.reset_index(drop=True), how='inner', on='USER_ID')
    d2 = df.merge(right=CU1M.reset_index(drop=True), how='inner', on='CUST_ID')
    
    logging.info("User id cust_id mapping data extraction completed")
    return d,d1,d2 

vishwaUserCustTrack,vishwaUserCustTrack1M,vishwaCustUserTrack1M=get_raw_vishwaUserCustTrack(connection_182,vishDisUser11,vishDisUser1M,vishUserCust,vishDisCust11,vishDisCust1M)

In [272]:
print(vishwaUserCustTrack.shape)
print(vishwaUserCustTrack1M.shape)
print(vishwaCustUserTrack1M.shape)

(299556, 2)
(7465, 2)
(804, 2)


In [106]:
userCustMap='SELECT USER_ID, \
CUST_ID, \
COUNT(*) OVER (PARTITION BY USER_ID) U_Cnt, \
COUNT(*) OVER (PARTITION BY CUST_ID) C_Cnt \
FROM tbaadm.svr_cust_id_table'
userCustMap

'SELECT USER_ID, CUST_ID, COUNT(*) OVER (PARTITION BY USER_ID) U_Cnt, COUNT(*) OVER (PARTITION BY CUST_ID) C_Cnt FROM tbaadm.svr_cust_id_table'

In [107]:
#Extracting all vishwa sheduled payment data 
def get_raw_userCustTrack(connection,queryString):
    logging.info("vishwa user cust data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("vishwa user cust  data extraction completed")
    return df 

userCustTrack=get_raw_userCustTrack(connection_182,userCustMap)

In [273]:
#Joining 4 tables to make vishwa Data frame 
vishwaDF = vishwaCustomer.merge(right=vishwaTransfer.reset_index(drop=True), how='left', on='USER_ID')
vishwaDF = vishwaDF.merge(right=vishwaPayment.reset_index(drop=True), how='left', on='USER_ID')
vishwaDF = vishwaDF.merge(right=vishwaSchdPayment.reset_index(drop=True), how='left', on='USER_ID')

In [274]:
vishwaDF['CUST_ID']=vishwaDF['CUST_ID'].astype(str)
print(vishwaDF.shape)
vishwaDF.head(3)

(308736, 24)


,USER_ID,CUST_ACCOUNT,NIC_NUMBER,CUST_REF_NO,BANK_CUST_FLG,V_GENDER,V_CIVIL_STATUS,V_CUST_DOB,V_CITY,CHARGE_PROFILE,...,V_OT_TXN_AMOUNT_LKR,V_RA_TXN_AMOUNT_LKR,V_RT_TXN_AMOUNT_LKR,V_MC_TXN_AMOUNT_LKR,V_TT_TXN_AMOUNT_LKR,V_TXN_CNT,V_PMNT_TRANSACTION_AMT_LKR,V_PMNT_COUNT,V_SCH_PMNT_AMOUNT_LKR,V_SCH_PMNT_CNT
0,menake,1.001e+11,761160532v,929100000003,Y,M,Married,25-04-1976,Thalahena,STF,...,5400106.01,689001.0,0.0,1000.0,20100.0,26.0,369616.11,50.0,NaN,NaN
1,wiggin,1.00105e+11,862102045V,929100000008,Y,M,Single,28-07-1986,kandy,STF,...,583679.18,0.0,0.0,0.0,17450.0,24.0,372179.17,23.0,88746.0,43.0
2,manjula,1.001e+11,732110577V,929100000010,Y,M,Married,29-07-1973,Ragama,STF,...,1104248.02,860280.0,0.0,28500.0,14500.0,30.0,319242.00,44.0,NaN,NaN


# Extracting Basic Customer data 

In [116]:
custQuery='select crm.core_cust_id as CUST_ID,lower(crm.uniqueidnumber),crm.gender,crm.staffflag,crm.card_holder, \
(EXTRACT(YEAR FROM SYSDATE)-crm.birth_year) CUST_AGE \
,crm.cust_type,crm.occupation,crm.city,(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-crm.relationshipopeningdate) BANK_CUST_AGE_DAYS, \
crm.seniorcitizen,crm.blacklisted,crm.negated,crm.suspended \
from crmuser.accounts crm \
inner join (select distinct cust_id from tbaadm.svr_cust_id_table) svr on crm.core_cust_id=svr.cust_id'

custQuery

"select crm.core_cust_id as CUST_ID,lower(crm.uniqueidnumber),crm.gender,crm.staffflag,crm.card_holder, (EXTRACT(YEAR FROM SYSDATE)-crm.birth_year) CUST_AGE ,crm.cust_type,crm.occupation,crm.city,(LAST_DAY(add_months(trunc(sysdate,'mm'),-1))-crm.relationshipopeningdate) BANK_CUST_AGE_DAYS, crm.seniorcitizen,crm.blacklisted,crm.negated,crm.suspended from crmuser.accounts crm inner join (select distinct cust_id from tbaadm.svr_cust_id_table) svr on crm.core_cust_id=svr.cust_id"

In [117]:
#Vishwa Customer related data extraction 
def get_raw_Customer(connection,queryString):
    logging.info(" Customer data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("customer data extraction completed")
    return df 
customer=get_raw_Customer(connection_182,custQuery)

In [186]:
customer['CUST_ID']=customer['CUST_ID'].astype(str)
print(customer.shape)
customer.head(3)

(312262, 14)


# Pawnning Data Extraction 

In [170]:
pawnCustomer='select lower(a.PTCUSID) AS P_NIC_NUMBER,SUM(case when ptstatus=\'R\' then 1 else 0 end) PTSTATUS_R, \
SUM(case when ptstatus=\'A\' then 1 else 0 end) PTSTATUS_A,SUM(case when ptstatus=\'C\' then 1 else 0 end) PTSTATUS_C, \
sum(PTDECLAREDVALUE) as PTVALUE \
from tbaadm.pawnticket a \
inner join tbaadm.advance b on a.ptnum=b.advptnum \
GROUP BY lower(a.ptcusid)'
pawnCustomer

"select lower(a.PTCUSID) AS P_NIC_NUMBER,SUM(case when ptstatus='R' then 1 else 0 end) PTSTATUS_R, SUM(case when ptstatus='A' then 1 else 0 end) PTSTATUS_A,SUM(case when ptstatus='C' then 1 else 0 end) PTSTATUS_C, sum(PTDECLAREDVALUE) as PTVALUE from tbaadm.pawnticket a inner join tbaadm.advance b on a.ptnum=b.advptnum GROUP BY lower(a.ptcusid)"

In [171]:
def get_raw_pawnCustomer(connection,queryString):
    logging.info("pawning customer data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("pawning customer data extraction completed")
    return df 

pawnCustomer=get_raw_pawnCustomer(connection_182,pawnCustomer)

In [46]:
#pawnCustomer=pd.read_csv("pawnCustomer.csv")
pawnCustomer.head(3)

,P_NIC_NUMBER,PTSTATUS_R,PTSTATUS_A,PTSTATUS_C,PTVALUE
0,911350998v,0,1,0,20472
1,396322,1,0,0,13379
2,511293790v,0,1,0,623767


# ATM Transaction data Extraction 

In [ ]:
txnATMQuery='select b.cust_id AS CUST_ID,Extract(month from added_date) AS ATM_TRXN_MONTH ,SUM(bill_amt) ATM_TRXN_BILL_AMT, \
SUM(trxn_amt) AS ATM_TRXN_AMT \
from stg.stg_swt_dx_trxn_all a \
inner join tbaadm.gam@stagelink b on a.orgn_acct_id=b.foracid \
where merc_cat_code in (\'6011\',\'6010\') and stat_code=1 and orgn_acct_id <> \'NA\' \
and added_date>= add_months(trunc(sysdate,\'mm\'),-6) AND added_date<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
GROUP BY b.cust_id,Extract(month from added_date)'
txnATMQuery

In [ ]:
def get_raw_txnATM(connection,queryString):
    logging.info("ATM Transactional data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("ATM Transactional data extraction completed")
    return df 

txnATM=get_raw_txnATM(connection_172,txnATMQuery)
#print(txnATM.head(5))

In [441]:
#txnATM=pd.read_csv("txnATM.csv")
txnATM['CUST_ID']=txnATM['CUST_ID'].astype(str)

'302159564'

# POS Transaction data extraction 

In [220]:
txnPOSQuery='select b.cust_id AS CUST_ID,Extract(month from added_date) AS POS_TRXN_MONTH ,SUM(bill_amt) POS_TRXN_BILL_AMT \
,SUM(trxn_amt) AS POS_TRXN_AMT \
from stg.stg_swt_dx_trxn_all a \
inner join tbaadm.gam@stagelink b on a.orgn_acct_id=b.foracid \
where merc_cat_code not in (\'6011\',\'6010\')  and orgn_acct_id <> \'NA\' \
AND merc_cat_code <> \'NA\' and merc_cat_code is not null \
and added_date>= add_months(trunc(sysdate,\'mm\'),-6) AND added_date<= LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
GROUP BY b.cust_id,Extract(month from added_date)'
txnPOSQuery

"select b.cust_id AS CUST_ID,Extract(month from added_date) AS POS_TRXN_MONTH ,SUM(bill_amt) POS_TRXN_BILL_AMT ,SUM(trxn_amt) AS POS_TRXN_AMT from stg.stg_swt_dx_trxn_all a inner join tbaadm.gam@stagelink b on a.orgn_acct_id=b.foracid where merc_cat_code not in ('6011','6010')  and orgn_acct_id <> 'NA' AND merc_cat_code <> 'NA' and merc_cat_code is not null and added_date>= add_months(trunc(sysdate,'mm'),-6) AND added_date<= LAST_DAY(add_months(trunc(sysdate,'mm'),-1)) GROUP BY b.cust_id,Extract(month from added_date)"

In [ ]:
def get_raw_txnPOS(connection,queryString):
    logging.info("POS Transactional data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("POS Transactional data extraction completed")
    return df 

txnPOS=get_raw_txnPOS(connection_172,txnPOSQuery)

In [340]:
#Consider this when reading from excel file 
#txnPOS=pd.read_csv("txnPOS.csv")
#CUST_ID=txnPOS['CUST_ID'].fillna(0).astype(int).astype(str)
#txnPOS['CUST_ID']=CUST_ID

In [345]:
txnPS['CUST_ID']=txnPOS['CUST_ID'].astype(str)
txnPOS.head(5)
#txnPOS.dtypes

,CUST_ID,POS_TRXN_MONTH,POS_TRXN_BILL_AMT,POS_TRXN_AMT
0,302139509,5,4125777,102274996
1,300920854,5,690825,4287
2,302291696,5,144965,900
3,303271529,5,636052,550299
4,302446992,5,620200,620200


In [337]:
txnPOS.dtypes

CUST_ID              object
POS_TRXN_MONTH        int64
POS_TRXN_BILL_AMT     int64
POS_TRXN_AMT          int64
dtype: object

# ATM and POS Aggregration

In [245]:
txnATMSum=txnATM[['CUST_ID','ATM_TRXN_BILL_AMT']].groupby(['CUST_ID']).mean()
txnATMSum['CUST_ID']=txnATMSum.index
txnATMSum['CUST_ID']=txnATMSum['CUST_ID'].astype(str)
txnATMSum.head(3)

txnPOSSum=txnPOS[['CUST_ID','POS_TRXN_BILL_AMT']].groupby(['CUST_ID']).mean()
txnPOSSum['CUST_ID']=txnPOSSum.index
txnPOSSum['CUST_ID']=txnPOSSum['CUST_ID'].astype(str)
txnPOSSum.head(3)

# Fixed deposit data extraction 

In [ ]:
fdQuery='select a.cust_id, \
sum(case when acct_cls_flg=\'Y\' then 1 else 0 end) FD_CLS_CNT,sum(case when acct_cls_flg=\'N\' then 1 else 0 end) FD_ACTV_CNT, \
SUM(CLR_BAL_AMT) FD_AMOUNT_SUM \
from tbaadm.gam@stagelink a \
left join DWH.dim_dayend_product b on a.schm_code=b.schm_code \
where lower(product_sub_type) like \'%fixed%\' \
GROUP BY a.cust_id'
fdQuery

In [ ]:
def get_raw_fd(connection,queryString):
    logging.info("Fixed deposit data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("Fixed deposit data extraction completed")
    return df 

fd=get_raw_fd(connection_172,fdQuery)

In [266]:
fd['CUST_ID']=fd['CUST_ID'].astype(str)
fd.head(3)

,CUST_ID,FD_CLS_CNT,FD_ACTV_CNT,FD_AMOUNT_SUM
0,200468341,61,5,4200000.0
1,200286004,43,0,0.0
2,200038790,23,4,1839208.3


# ODA Data extraction

In [ ]:
odQuery='select a.cust_id, \
sum(case when acct_cls_flg=\'Y\' then 1 else 0 end) ODA_CLS_CNT, \
sum(case when acct_cls_flg=\'N\' then 1 else 0 end) ODA_ACTV_CNT, \
SUM(CLR_BAL_AMT) ODA_AMOUNT_TOT \
from tbaadm.gam@stagelink a \
WHERE SCHM_TYPE=\'ODA\' \
GROUP BY a.cust_id'
odQuery

In [ ]:
def get_raw_oda(connection,queryString):
    logging.info("over draft data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("over draft data extraction completed")
    return df 

oda=get_raw_oda(connection_172,odQuery)

In [70]:
oda['CUST_ID']=oda['CUST_ID'].astype(str)
oda.head(3)

,CUST_ID,ODA_CLS_CNT,ODA_ACTV_CNT,ODA_AMOUNT_TOT
0,26972,1,0,0.0
1,PW004,1,0,0.0
2,BROPEN,2,0,0.0


# Loan Data Extraction 

In [37]:
loanQuery='select A.CUST_ID, \
SUM(case when acct_cls_flg=\'Y\' then 1 else 0 end) INACTV_LOAN_CNT, \
SUM(case when acct_cls_flg=\'N\' then 1 else 0 end) ACTV_LOAN_CNT, \
SUM(case when acct_cls_flg=\'Y\' then a.SANCT_LIM else 0 end) INACTV_LOAN_LIM_TOT, \
SUM(case when acct_cls_flg=\'N\' then a.SANCT_LIM else 0 end) ACTV_LOAN__LIM_TOT, \
SUM(CLR_BAL_AMT) AS TOT_BAL, \
MAX(a.SANCT_LIM) MAX_LOAN_LIM, \
MIN(a.SANCT_LIM) MIN_LOAN__LIM, \
(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-MAX(a.ACCT_OPN_DATE)) LAST_LOAN_AGE_DAYS, \
(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-MIN(a.ACCT_OPN_DATE)) FIRST_LOAN_AGE_DAYS, \
sum(case when lower(our_product_group) like \'%staff%\' then 1 else 0 end) STAFF_LOAN_CNT, \
sum(case when lower(our_product_group) like \'%short%\' then 1 else 0 end) ST_LOAN_CNT, \
sum(case when lower(our_product_group) like \'%medium%\' then 1 else 0 end) MT_LOAN_CNT, \
sum(case when lower(our_product_group) like \'%long%\' then 1 else 0 end) LT_LOAN_CNT, \
sum(case when lower(our_product_group) like \'%housing%\' then 1 else 0 end) HOUSING_LOAN_CNT \
from tbaadm.gam@stagelink a \
inner join (select distinct schm_code,our_product_group from stg.f10_ccu_adv_portfolio) b on a.schm_code=b.schm_code \
and lower(b.our_product_group) like \'%loan%\' \
group by a.cust_id '
loanQuery

"select A.CUST_ID, SUM(case when acct_cls_flg='Y' then 1 else 0 end) INACTV_LOAN_CNT, SUM(case when acct_cls_flg='N' then 1 else 0 end) ACTV_LOAN_CNT, SUM(case when acct_cls_flg='Y' then a.SANCT_LIM else 0 end) INACTV_LOAN_LIM_TOT, SUM(case when acct_cls_flg='N' then a.SANCT_LIM else 0 end) ACTV_LOAN__LIM_TOT, SUM(CLR_BAL_AMT) AS TOT_BAL, MAX(a.SANCT_LIM) MAX_LOAN_LIM, MIN(a.SANCT_LIM) MIN_LOAN__LIM, (LAST_DAY(add_months(trunc(sysdate,'mm'),-1))-MAX(a.ACCT_OPN_DATE)) LAST_LOAN_AGE_DAYS, (LAST_DAY(add_months(trunc(sysdate,'mm'),-1))-MIN(a.ACCT_OPN_DATE)) FIRST_LOAN_AGE_DAYS, sum(case when lower(our_product_group) like '%staff%' then 1 else 0 end) STAFF_LOAN_CNT, sum(case when lower(our_product_group) like '%short%' then 1 else 0 end) ST_LOAN_CNT, sum(case when lower(our_product_group) like '%medium%' then 1 else 0 end) MT_LOAN_CNT, sum(case when lower(our_product_group) like '%long%' then 1 else 0 end) LT_LOAN_CNT, sum(case when lower(our_product_group) like '%housing%' then 1 else 0 en

In [ ]:
def get_raw_loan(connection,queryString):
    logging.info("loan data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("loan data extraction completed")
    return df 

loan=get_raw_loan(connection_172,loanQuery)

In [71]:
loan['CUST_ID']=loan['CUST_ID'].astype(str)
loan.head(5)

,CUST_ID,INACTV_LOAN_CNT,ACTV_LOAN_CNT,INACTV_LOAN_LIM_TOT,ACTV_LOAN__LIM_TOT,TOT_BAL,MAX_LOAN_LIM,MIN_LOAN__LIM,LAST_LOAN_AGE_DAYS,FIRST_LOAN_AGE_DAYS,STAFF_LOAN_CNT,ST_LOAN_CNT,MT_LOAN_CNT,LT_LOAN_CNT,HOUSING_LOAN_CNT
0,200261078,9,16,4589950.0,7074000.0,-4.628907e+06,2200000.0,45000.0,21,7574,25,0,0,0,0
1,200121228,3,0,47000.0,0.0,0.000000e+00,30000.0,7000.0,5615,7384,0,0,3,0,0
2,200062899,1,1,1200000.0,0.0,1.087345e+05,1200000.0,0.0,5619,10193,1,0,0,0,1
3,NaN,4650,84442,0.0,0.0,-4.188258e+11,0.0,0.0,-2,7301,44546,0,0,0,0
4,200147327,1,0,100000.0,0.0,0.000000e+00,100000.0,100000.0,5615,5615,0,0,1,0,0


# Lease data extraction 

In [23]:
leaseQuery='select A.CUST_ID, \
SUM(case when acct_cls_flg=\'Y\' then 1 else 0 end) INACTV_LEASE_CNT, \
SUM(case when acct_cls_flg=\'N\' then 1 else 0 end) ACTV_LEASE_CNT, \
SUM(case when acct_cls_flg=\'Y\' then a.SANCT_LIM else 0 end) INACTV_LEASE_LIM_TOT, \
SUM(case when acct_cls_flg=\'N\' then a.SANCT_LIM else 0 end) ACTV_LEASE__LIM_TOT, \
SUM(CLR_BAL_AMT) AS TOT_BAL, \
MAX(a.SANCT_LIM) MAX_LEASE_LIM, \
MIN(a.SANCT_LIM) MIN_LEASE__LIM, \
(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-MAX(a.ACCT_OPN_DATE)) LAST_LEASE_AGE_DAYS, \
(LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1))-MIN(a.ACCT_OPN_DATE)) FIRST_LEASE_AGE_DAYS \
from tbaadm.gam@stagelink a \
inner join (select distinct schm_code,our_product_group from stg.f10_ccu_adv_portfolio) b on a.schm_code=b.schm_code \
and lower(b.our_product_group) like \'%lease%\' \
group by a.cust_id'

leaseQuery

"select A.CUST_ID, SUM(case when acct_cls_flg='Y' then 1 else 0 end) INACTV_LEASE_CNT, SUM(case when acct_cls_flg='N' then 1 else 0 end) ACTV_LEASE_CNT, SUM(case when acct_cls_flg='Y' then a.SANCT_LIM else 0 end) INACTV_LEASE_LIM_TOT, SUM(case when acct_cls_flg='N' then a.SANCT_LIM else 0 end) ACTV_LEASE__LIM_TOT, SUM(CLR_BAL_AMT) AS TOT_BAL, MAX(a.SANCT_LIM) MAX_LEASE_LIM, MIN(a.SANCT_LIM) MIN_LEASE__LIM, (LAST_DAY(add_months(trunc(sysdate,'mm'),-1))-MAX(a.ACCT_OPN_DATE)) LAST_LEASE_AGE_DAYS, (LAST_DAY(add_months(trunc(sysdate,'mm'),-1))-MIN(a.ACCT_OPN_DATE)) FIRST_LEASE_AGE_DAYS from tbaadm.gam@stagelink a inner join (select distinct schm_code,our_product_group from stg.f10_ccu_adv_portfolio) b on a.schm_code=b.schm_code and lower(b.our_product_group) like '%lease%' group by a.cust_id"

In [24]:
def get_raw_lease(connection,queryString):
    logging.info("lease data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("lease data extraction completed")
    return df 

lease=get_raw_lease(connection_172,leaseQuery)

In [26]:
lease['CUST_ID']=lease['CUST_ID'].astype(str)
lease.head(5)

,CUST_ID,INACTV_LEASE_CNT,ACTV_LEASE_CNT,INACTV_LEASE_LIM_TOT,ACTV_LEASE__LIM_TOT,TOT_BAL,MAX_LEASE_LIM,MIN_LEASE__LIM,LAST_LEASE_AGE_DAYS,FIRST_LEASE_AGE_DAYS
0,None,2325,42221,0.00,0.0,-2.079738e+11,0.00,0.00,-2,7301
1,300110761,1,0,1672147.68,0.0,0.000000e+00,1672147.68,1672147.68,6201,6201
2,200418947,1,0,64482.84,0.0,0.000000e+00,64482.84,64482.84,6221,6221
3,200248408,1,0,3987613.44,0.0,0.000000e+00,3987613.44,3987613.44,7812,7812
4,200310946,2,0,1638143.04,0.0,0.000000e+00,1537237.92,100905.12,4712,6829


# Transaction credit data extraction 

In [ ]:
july='select a.cust_id, \
SUM(a.TRAN_AMT ) TOT_TXN_CREDIT, \
SUM(CASE WHEN a.TRAN_PARTICULAR LIKE \'%SLIP%\' THEN a.TRAN_AMT ELSE 0 END ) AS SLIP_TXN_CREDIT, \
SUM(CASE WHEN (h.remarks LIKE \'%CPAY%\' OR (UPPER(TRIM(a.TRAN_PARTICULAR)) LIKE \'%SALAR%\' AND h.remarks =\'SLIPS\')) THEN a.TRAN_AMT ELSE 0 END ) AS CPAY_TXN_CREDIT \
from tbaadm.htd a \
inner join tbaadm.hth h on a.tran_id=h.tran_id and a.tran_date=h.tran_date \
where extract(year from a.TRAN_DATE)=\'2018\' and extract(month from a.tran_date)=7 and a.cust_id is not null \
and a.del_flg != \'Y\' AND a.part_tran_type <> \'D\' AND a.pstd_flg = \'Y\' AND h.del_flg != \'Y\' \
group by a.cust_ID'
july

In [ ]:
def get_raw_txnCredit(connection,queryString):
    logging.info("Credit transaction data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("Credit transaction data extraction completed")
    return df 

txnCredit=get_raw_txnCredit(connection_182,txnCreditQuery)
print(txnCredit.head(5))

In [ ]:
txnCredit=pd.read_csv("july.csv")
txnCredit['CUST_ID']=txnCredit['CUST_ID'].astype(str)
txnCredit.head()

# Transaction debit data extraction 

In [ ]:
july='select a.cust_id, \
SUM(a.TRAN_AMT) TOT_TXN_DEBIT \
from tbaadm.htd a \
inner join tbaadm.hth h on a.tran_id=h.tran_id and a.tran_date=h.tran_date \
where extract(year from a.TRAN_DATE)=\'2018\' and extract(month from a.tran_date)=7 and a.cust_id is not null \
and a.del_flg != \'Y\' AND a.part_tran_type=\'D\' AND a.pstd_flg = \'Y\' AND h.del_flg != \'Y\' \
group by a.cust_ID'
july

In [ ]:
def get_raw_txnDebit(connection,queryString):
    logging.info("Debit transaction data extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("Debit transaction data extraction completed")
    return df 

txnCredit=get_raw_txnDebit(connection_182,txnDebitQuery)
print(txnCredit.head(5))

In [ ]:
txnDebit=pd.read_csv("julyD.csv")
txnDebit['CUST_ID']=txnDebit['CUST_ID'].astype(str)
txnDedit.head()

# Credit card data extraction

In [34]:
ccQuery='select distinct lower(p.ssnumber) as NIC_NUMBER,max(a.creditlimit) as CreditLimit \
from tbaadm.cardx c \
left join tbaadm.ctransactions ct on c.serno=ct.cardserno \
inner join tbaadm.people p on c.peopleserno=p.serno \
inner join tbaadm.caccounts a on c.caccserno=a.serno \
group by lower(p.ssnumber)'
ccQuery

'select distinct lower(p.ssnumber) as NIC_NUMBER,max(a.creditlimit) as CreditLimit from tbaadm.cardx c left join tbaadm.ctransactions ct on c.serno=ct.cardserno inner join tbaadm.people p on c.peopleserno=p.serno inner join tbaadm.caccounts a on c.caccserno=a.serno group by lower(p.ssnumber)'

In [35]:
def get_raw_ccDetails(connection,queryString):
    logging.info("credit card details extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("credi card details extraction completed")
    return df 

cc=get_raw_ccDetails(connection_182,ccQuery)

In [59]:
cc.head(3)

,NIC_NUMBER,CREDITLIMIT
0,***********,2000000.0
1,-,750000.0
2,----,100000.0


# Joint account Cust_ids 

In [53]:
jointAcctQuery='select distinct cust_id \
from tbaadm.aas \
where acid in \
  (select acid from tbaadm.aas \
   group by acid having count(*)>1)'
jointAcctQuery

'select distinct cust_id from tbaadm.aas where acid in   (select acid from tbaadm.aas    group by acid having count(*)>1)'

In [54]:
def get_raw_jointAcct(connection,queryString):
    logging.info("Joint account details extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("Joint account details extraction completed")
    return df 

jointAcct=get_raw_jointAcct(connection_182,jointAcctQuery)

In [57]:
jointAcct.to_csv("jointAcct.csv")
jointAcct.head(3)

,CUST_ID
0,2
1,84
2,XSET


# Data Frame Preparation 

In [445]:
df = vishwaDF.merge(right=vishwaUserCustTrack.reset_index(drop=True), how='inner', on='USER_ID')
df1 = df.merge(right=customer.reset_index(drop=True), how='left', on='CUST_ID')
df2=df1.merge(right=pawnCustomer.reset_index(drop=True), how='left', left_on='UNIQUEIDNUMBER',right_on='P_NIC_NUMBER')
df3=df2.merge(right=txnATMSum.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df4=df3.merge(right=txnPOSSum.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df5=df4.merge(right=fd.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df6=df5.merge(right=oda.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df7=df6.merge(right=loan.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df9=df8.merge(right=lease.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
df10=df9.merge(right=cc.reset_index(drop=True), how='left', left_on='UNIQUEIDNUMBER',right_on='NIC_NUMBER')

#df9=df8.merge(right=txnDedit.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')


#lp=df6.merge(right=loan.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')

In [ ]:
df12=df11.merge(right=txa.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')

In [61]:
#Tempory 

df7=pd.read_csv("lp.csv")
df7['CUST_ID']=df7['CUST_ID'].astype(str)
julyCredit=pd.read_csv("JULY_TRXNS.csv")
julyCredit['CUST_ID']=julyCredit['CUST_ID'].astype(str)
df8=df7.merge(right=julyCredit.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
#df9=df8.merge(right=txnDedit.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')
#df8.to_csv("lp.csv",index=False)

#df9=pd.read_csv("lp.csv")
#df10=df9.merge(right=cc.reset_index(drop=True), how='left', left_on='UNIQUEIDNUMBER',right_on='NIC_NUMBER')
#df10.to_csv("lp.csv")

#df11=df9.merge(right=cc.reset_index(drop=True), how='left', left_on='UNIQUEIDNUMBER',right_on='NIC_NUMBER')
#df10.to_csv("lp.csv")


julyDebit=pd.read_csv("JULYD.csv")
lp=pd.read_csv("lp.csv")
julyDebit['CUST_ID']=julyDebit['CUST_ID'].astype(str)
lp['CUST_ID']=lp['CUST_ID'].astype(str)
df11=lp.merge(right=julyDebit.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')


/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [62]:
julyDebit=pd.read_csv("JULYD.csv")
lp=pd.read_csv("lp.csv")
julyDebit['CUST_ID']=julyDebit['CUST_ID'].astype(str)
lp['CUST_ID']=lp['CUST_ID'].astype(str)
df11=lp.merge(right=julyDebit.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')

/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Feedback DF Preparation 

In [64]:
fbQuery='select distinct cust_id \
from tbaadm.gam \
where schm_code=\'VIPAF\''

In [89]:
def get_raw_feedBack(connection,queryString):
    logging.info("FGeedback data extraction started")
    df1 = pd.read_sql(queryString,con=connection)
    df2 = pd.read_csv("feedback.csv")
    df1['Comment']=1
    
    df3=df2[(~df2.CUST_ID.isin(df1.CUST_ID))]
    fb=df1.append(df3, ignore_index=True,sort=False)
    fb.set_index('CUST_ID')
    logging.info("Feedback data extraction completed")
    return fb

fb=get_raw_feedBack(connection_182,fbQuery)
fb.shape

(106, 2)

# Transaction

In [91]:
tx=pd.read_csv("Tx.csv")

In [94]:
tx['cpayFlag']=0
tx.loc[(tx.CPAY>75000) | (tx.SLIP >75000),'cpayFlag']=1

In [98]:
txa = tx.groupby(['CUST_ID']).agg({'C':'mean','D':'mean','cpayFlag':sum})

In [100]:
df11=df10.merge(right=txnDebit.reset_index(drop=True), how='left', left_on='CUST_ID',right_on='CUST_ID')

(299223, 80)

In [104]:
txa.to_csv("transaction.csv",index=True)

# Writing Dataframes into CSV

In [448]:
#Writing data to CSV files 
def writeCSV():
    logging.info("Writing to CSV started")
    vishwaCustomer.to_csv("vishwaCustomer.csv", index=False)
    vishwaTransfer.to_csv("vishwaTransfer.csv", index=False)
    vishwaPayment.to_csv("vishwaPayment.csv", index=False)
    vishwaSchdPayment.to_csv("vishwaSchdPayment.csv", index=False)
    vishwaUserCustTrack.to_csv("vishwaUserCustTrack.csv",index=False)
    vishwaUserCustTrack1M.to_csv("vishwaUserCustTrack1M.csv",index=False)
    vishwaCustUserTrack1M.to_csv("vishwaCustUserTrack1M.csv",index=False)
    vishwaDF.to_csv("vishwaDF.csv",index=False)
    customer.to_csv("customer.csv",index=False)
    pawnCustomer.to_csv("pawnCustomer.csv",index=False)
    txnATM.to_csv("txnATM.csv",index=False)
    txnPOS.to_csv("txnPOS.csv",index=False)
    txnATMSum.to_csv("txnATMSum.csv",index=False)
    txnPOSSum.to_csv("txnPOSSum.csv",index=False)
    fd.to_csv("fd.csv",index=False)
    oda.to_csv("oda.csv",index=False)
    loan.to_csv("loan.csv",index=False)
    cc.to_csv("cc.csv",index=False)
    cc.to_csv("lease.csv",index=False)
    lp.to_csv("lp.csv",index=False)
    fb.to_csv("fb.csv",index=False)
    userCustTrack.to_csv("userCustTrack.csv",index=False)
    logging.info("Writing to CSV Completed")
writeCSV()

# Customer demographical data preparation 

In [34]:
crmQuery='select \
CORE_CUST_ID as CUST_ID,UNIQUEIDNUMBER AS CRM_NIC1,UNIQUEID CRM_NIC2,CUST_lAST_NAME AS CRM_CUST_NAME,GENDER AS CRM_GENDER, \
(EXTRACT(YEAR FROM SYSDATE)-BIRTH_YEAR) CRM_AGE,OCCUPATION CRM_OCCUPATION, \
PHONE CRM_PHONE1,PHONE_HOME CRM_PHONE2,PHONE_HOME2 CRM_PHONE3,PHONE_CELL CRM_PHONE4,PREFERREDPHONE CRM_PHONE5 , EMAIL CRM_EMAIL1, \
EMAIL_PALM CRM_EMAIL2,EMAIL_HOME CRM_EMAIL3,PRIMARY_SOL_ID CRM_BRANCH_ID,CONCAT(CONCAT(ADDRESS_LINE1,ADDRESS_LINE2), \
ADDRESS_LINE3) AS CRM_ADDRESS \
,STATE AS CRM_STATE \
from CRMUSER.ACCOUNTS'


In [35]:
def get_raw_crmUser(connection,queryString):
    logging.info("crmuser accounts details extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("crmuser accounts details extraction completed")
    return df 

crmUser=get_raw_crmUser(connection_182,crmQuery)

In [36]:
vQuery='select USER_ID as USER_ID,NIC_NUMBER AS V_NIC_NUMBER,PP_NUMBER AS V_PP_NUMBER,EMAIL_ADDR AS V_EMAIL, \
CONCAT(CUST_OTHER_NAMES , CUST_SURNAME) AS V_CUST_NAME,CONCAT(COMM_ADDR1,COMM_ADDR2) AS V_ADDRESS, \
COMM_TEL_NO AS V_PHONE1,COMM_MOBILE_NO AS V_PHONE2 \
from tbaadm.svr_user_master_table'


In [37]:
def get_raw_vUser(connection,queryString):
    logging.info("vishwa user accounts details extraction started")
    df = pd.read_sql(queryString,con=connection)
    logging.info("vishwa user accounts details extraction completed")
    return df 

vUser=get_raw_vUser(connection_182,vQuery)

In [38]:
userCustTrack=pd.read_csv("userCustTrack.csv",dtype={'USER_ID':'object','CUST_ID':'object'})
df0=userCustTrack[(userCustTrack.U_CNT==1) & (userCustTrack.C_CNT==1)]
df = vUser.merge(right=df0.reset_index(drop=True), how='inner',on='USER_ID')
df1= df.merge(right=crmUser.reset_index(drop=True), how='left', on='CUST_ID')

In [39]:
df1.to_csv("custDemo.csv",index=False)

In [ ]:
#Transaction 

In [60]:
que='select a.cust_id,a.tran_amt,a.tran_particular,g.foracid,h.remarks,a.part_tran_type \
from tbaadm.htd a \
inner join tbaadm.hth h on a.tran_id=h.tran_id and a.tran_date=h.tran_date \
inner join tbaadm.gam g on a.acid=g.acid \
where a.tran_date>=add_months(trunc(sysdate,\'mm\'),-1) \
and a.tran_date<add_months(trunc(sysdate,\'mm\'),-1)+1 \
and a.cust_id is not null and a.del_flg != \'Y\' AND a.pstd_flg = \'Y\' AND h.del_flg != \'Y\' \
'
que

"select a.cust_id,a.tran_amt,a.tran_particular,g.foracid,h.remarks,a.part_tran_type from tbaadm.htd a inner join tbaadm.hth h on a.tran_id=h.tran_id and a.tran_date=h.tran_date inner join tbaadm.gam g on a.acid=g.acid where a.tran_date>=add_months(trunc(sysdate,'mm'),-1) and a.tran_date<add_months(trunc(sysdate,'mm'),-1)+1 and a.cust_id is not null and a.del_flg != 'Y' AND a.pstd_flg = 'Y' AND h.del_flg != 'Y' "

In [61]:
%%time
df = pd.read_sql(que,con=connection_182)

CPU times: user 1.67 s, sys: 286 ms, total: 1.96 s
Wall time: 12.3 s


In [33]:
df[(df.FORACID.str[:1] == '0')|(df.FORACID.str[:1] == '1')].shape

(496474, 7)

In [68]:
#dt=pd.read_csv("data.csv")
#vishwaCust=dt['CUST_ID']

(302258,)

In [71]:
import calendar
import datetime
now = datetime.datetime.now()
print(calendar.monthrange(now.year, now.month)[1])


31


datetime.datetime(2018, 8, 24, 15, 7, 27, 13626)

In [83]:
df.head(1)

,CUST_ID,TRAN_AMT,TRAN_PARTICULAR,FORACID,REMARKS,PART_TRAN_TYPE
0,200312922,295.0,LN-INS73000144,100253287002,Loan Auto Recovery,D


In [87]:
df=df.fillna({'TRAN_PARTICULAR':'NA','REMARKS':'NA'})
df['tx_cat'] = df.apply(setTxCode,axis=1)

In [90]:
df = df[(df.FORACID.str[:1] == '0')|(df.FORACID.str[:1] == '1')]
df = df[['CUST_ID','TRAN_AMT','PART_TRAN_TYPE','tx_cat']]

In [95]:
df = df.groupby(['CUST_ID','PART_TRAN_TYPE','tx_cat']).agg({'TRAN_AMT':sum})

In [97]:
df = df.reset_index()

In [106]:
sys.getsizeof(df)

2952371416

In [105]:
df = pd.concat([df,df],axis=0)

In [107]:
%%time
df.to_csv('test.csv')

CPU times: user 46.5 s, sys: 532 ms, total: 47.1 s
Wall time: 47.1 s


In [82]:
def setTxCode(x):
    if ('SLIP' in x.TRAN_PARTICULAR):
        y = 'S'
    elif (('CPAY' in x.REMARKS) or (('SALAR' in x.TRAN_PARTICULAR) and (x.REMARKS == 'SLIP'))):
        y = 'Z'
    else :
        y = 'O'
    return y

In [75]:
que='select a.cust_id,a.tran_amt,a.tran_particular,g.foracid,h.remarks,a.part_tran_type \
from tbaadm.htd a \
inner join tbaadm.hth h on a.tran_id=h.tran_id and a.tran_date=h.tran_date \
inner join tbaadm.gam g on a.acid=g.acid \
where a.tran_date>=add_months(trunc(sysdate,\'mm\'),-1) \
and a.tran_date<=LAST_DAY(add_months(trunc(sysdate,\'mm\'),-1)) \
and a.cust_id is not null and a.del_flg != \'Y\' AND a.pstd_flg = \'Y\' AND h.del_flg != \'Y\' \
and to_date(a.tran_date)={0}'

In [ ]:
import calendar
import datetime

now=datetime.datetime.now()
days=calendar.monthrange(now.year, now.month)[1]

df=pd.DataFrame

for x in range(days):
    now=datetime.datetime.now() 
    y=now.year
    m=now.month
    d=x
    date=datetime.datetime(year=y, month=m, day=x)
    dfTemp= pd.read_sql(que.format(date),con=connection_182)

    dfTemp=dfTemp.fillna({'TRAN_PARTICULAR':'NA','REMARKS':'NA'})
    dfTemp['tx_cat'] = dfTemp.apply(setTxCode,axis=1)
    
    dfTemp = dfTemp[(dfTemp.FORACID.str[:1] == '0')|(dfTemp.FORACID.str[:1] == '1')]
    dfTemp = dfTemp[['CUST_ID','TRAN_AMT','PART_TRAN_TYPE','tx_cat']] 
    dfTemp = dfTemp.groupby(['CUST_ID','PART_TRAN_TYPE','tx_cat']).agg({'TRAN_AMT':sum})
    df = df.reset_index()
    
    df=pd.concat(df,dfTemp,axis=0)
    


df # dailyTx
df = vdf and df inner join
cdf concat df